In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers

from sklearn.metrics import roc_auc_score

In [2]:
%run data_to_train.ipynb

print('---------- training: ANN')

---------- read_data
train data shape:	 (1526659, 376)
test data shape:	 (10, 375)
train data shape:	 (1526659, 267)
test data shape:	 (10, 266)
---------- clean_data
---------- data_to_train
---------- training: ANN


In [6]:
def create_deep_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)

    # Create multiple dense layers as specified by the hidden_units list
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i], activation=None,kernel_regularizer=regularizers.l2(0.01))(x)  # Use None for activation here since we're applying it after batch norm
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)  # Notice dropout_rates[i + 1] because of initial dropout after input layer

    # Output layer
    x = tf.keras.layers.Dense(num_labels, activation=None,kernel_regularizer=regularizers.l2(0.01))(x)  # Use None for activation here since we're applying it after
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    # Create model
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing), 
                  metrics=[tf.keras.metrics.AUC(name='AUC')])
    
    return model

In [7]:
import random

random_seed = 12345
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [8]:
ckp_path = './saved/NNModel.hdf5'
learning_rate = 1e-3
label_smoothing = 0.01
batch_size = 32
hidden_units = [64, 128, 64, 32]
dropout_rates = [0.1, 0.1, 0.1,0.1, 0.1]

# First Training Fit

In [ ]:
## If your GPU is supporting you can leverage this code
# from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy('mixed_float16') # Set global data policy to mixed precision.

In [9]:
# Model creation
model = create_deep_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, label_smoothing, learning_rate)

# Callbacks
rlr = ReduceLROnPlateau(monitor='val_AUC', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='max')
ckp = ModelCheckpoint(ckp_path, monitor='val_AUC', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
es = EarlyStopping(monitor='val_AUC', min_delta=1e-4, patience=3, mode='max', restore_best_weights=True, verbose=1)

# Model fitting
history = model.fit(X_train, y_train,
                    validation_data=(X_valid, y_valid),
                      epochs=100, 
                      batch_size=batch_size, 
                      callbacks=[rlr, ckp, es], 
                      verbose=1)

Epoch 1/100
40710/40711 [============================>.] - ETA: 0s - loss: 0.1768 - AUC: 0.7194
Epoch 1: val_AUC improved from -inf to 0.65340, saving model to NNModel.hdf5
40711/40711 [==============================] - 459s 11ms/step - loss: 0.1768 - AUC: 0.7194 - val_loss: 0.1256 - val_AUC: 0.6534 - lr: 0.0010
Epoch 2/100
40710/40711 [============================>.] - ETA: 0s - loss: 0.1551 - AUC: 0.7448
Epoch 2: val_AUC did not improve from 0.65340
40711/40711 [==============================] - 452s 11ms/step - loss: 0.1551 - AUC: 0.7448 - val_loss: 589.7628 - val_AUC: 0.5260 - lr: 0.0010
Epoch 3/100
40707/40711 [============================>.] - ETA: 0s - loss: 0.1547 - AUC: 0.7475
Epoch 3: val_AUC did not improve from 0.65340
40711/40711 [==============================] - 462s 11ms/step - loss: 0.1547 - AUC: 0.7475 - val_loss: 27102.5938 - val_AUC: 0.5264 - lr: 0.0010
Epoch 4/100
40711/40711 [==============================] - ETA: 0s - loss: 0.1544 - AUC: 0.7471
Epoch 4: ReduceLRO

# Second Model Fitting

In [10]:
# Load the best weights
model.load_weights(ckp_path)

# Fine-tuning on validation data
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate / 100),
              loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing), 
              metrics=[tf.keras.metrics.AUC(name='AUC')])
hisotry2 = model.fit(X_valid, y_valid, epochs=5, batch_size=batch_size, verbose=1)

# Save the fine-tuned weights
model.save_weights(ckp_path)

# Evaluation
# Here you should use your validation set or a separate test set
predicted_valid = model.predict(X_valid, batch_size=batch_size * 4).ravel()
auc_score = roc_auc_score(y_valid, predicted_valid)
print(f'ROC AUC Score after fine-tuning: {auc_score}')

Epoch 1/5
6998/6998 [==============================] - 70s 10ms/step - loss: 0.1180 - AUC: 0.7851
Epoch 2/5
6998/6998 [==============================] - 82s 12ms/step - loss: 0.1158 - AUC: 0.7925
Epoch 3/5
6998/6998 [==============================] - 73s 10ms/step - loss: 0.1149 - AUC: 0.7968
Epoch 4/5
6998/6998 [==============================] - 72s 10ms/step - loss: 0.1144 - AUC: 0.7981
Epoch 5/5
1750/1750 [==============================] - 4s 2ms/step
ROC AUC Score after fine-tuning: 0.8223442577685152


In [11]:
# Cleanup
tf.keras.backend.clear_session()
gc.collect()

3040